## Keras 

Keras 是一个用于构建和训练深度学习模型的高阶 API，可用于快速设计原型、高级研究和生产，具有以下三个优势：

- 方便使用
- 模型块化可组合
- 易于扩展

### `tf.keras`

`tf.keras` 是 TensorFlow 对 Keras API 规范的实现。TensorFlow 在实现这些 API 时，包含了对 **TensorFlow 特定功能** 的支持，如：Eager Execution, tf.data 管道 、 Estaimator 等。

In [19]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers

### 构建简单的模型

#### 序列模型

在 Keras 中，您可以通过组合层来构建模型。模型（通常）是由层构成的图。最常见的模型类型是层的堆叠：`tf.keras.Sequential` 模型。

#### 配置层

我们可以使用很多 `tf.keras.layers`，它们具有一些相同的构造函数参数：

- activation：设置层的激活函数。此参数由内置函数的名称指定，或指定为可调用对象。**默认情况下，系统不会应用任何激活函数。**
- kernel_initializer 和 bias_initializer：创建层权重（核和偏差）的初始化方案。此参数是一个名称或可调用对象，默认为 "Glorot uniform" 初始化器。
- kernel_regularizer 和 bias_regularizer：应用层权重（核和偏差）的正则化方案，例如 L1 或 L2 正则化。默认情况下，系统不会应用正则化函数。



### 训练和评估

#### 设置训练流程

构建好模型后，通过调用 `compile` 方法配置该模型的学习流程。

tf.keras.Model.compile 采用三个重要参数：

- optimizer：此对象会指定训练过程。从 tf.train 模块向其传递优化器实例，例如 tf.train.AdamOptimizer、tf.train.RMSPropOptimizer 或 tf.train.GradientDescentOptimizer。

- loss：要在优化期间最小化的函数。常见选择包括均方误差 (mse)、categorical_crossentropy 和 binary_crossentropy。损失函数由名称或通过从 tf.keras.losses 模块传递可调用对象来指定。

- metrics：用于监控训练。它们是 tf.keras.metrics 模块中的字符串名称或可调用对象。

#### 输入 Numpy 数据

**对于小型数据集**，可以使用内存中的 Numpy 数组训练和评估模型。

#### 输入 tf.data 数据集

而对于大型数据集和多设备训练。可以将 `tf.data.Dataset` 实例传递到 `fit` 方法中。利用 Datasets API 来训练。


#### 评估和预测

`tf.keras.Model.evaluate` 和 `tf.keras.Model.predict` 方法可以使用 NumPy 数据和 tf.data.Dataset。

### 构建高级模型

`tf.keras.Sequential` 模型是层的简单堆叠，无法表示任意模型。使用 Keras 函数式 API 可以构建复杂的模型拓扑。

- 多输入模型，

- 多输出模型，

- 具有共享层的模型（同一层被调用多次），

- 具有非序列数据流的模型（例如，剩余连接）。

使用函数式 API 构建的模型具有以下特征：

- 层实例可调用并返回张量。
- 输入张量和输出张量用于定义 tf.keras.Model 实例。
- 此模型的训练方式和 Sequential 模型一样。

### 创建自己的模型

通过对 `tf.keras.Model` 的继承，可实现自定义的前向模型。在启用 Eager Execution 时，模型子类化特别有用，因为可以命令式地编写前向传播。


In [12]:
class MyModel(tf.keras.Model):
    
    def __init__(self, num_classes=10):
        super(MyModel, self).__init__(name='my_model')
        self.dense_1 = layers.Dense(32, activation='relu')
        self.dense_2 = layers.Dense(num_classes,
                                   activation='sigmoid')
    
    def call(self, inputs):
        # Define your forward pass here
        x = self.dense_1(inputs)
        return self.dense_2(x)
    
    def compute_output_shape(self, input_shape):
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.num_classes
        return tf.TensorShape(shape)

In [17]:
model = MyModel(num_classes=10)

# The compile step specifies the training configuration.
model.compile(optimizer=optimizers.RMSprop(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [22]:
data = np.random.random((10, 32))
labels = np.random.random((10, 10))

model.fit(data, labels, epochs=5)

Epoch 1/5
10/10 [==============================] - 0s 16ms/sample - loss: 11.1602 - accuracy: 0.1000
Epoch 2/5
10/10 [==============================] - 0s 175us/sample - loss: 11.1219 - accuracy: 0.2000
Epoch 3/5
10/10 [==============================] - 0s 352us/sample - loss: 11.0984 - accuracy: 0.3000
Epoch 4/5
10/10 [==============================] - 0s 618us/sample - loss: 11.0803 - accuracy: 0.3000
Epoch 5/5
10/10 [==============================] - 0s 356us/sample - loss: 11.0657 - accuracy: 0.3000


#### 自定义层

通过对 `tf.keras.layers.Layer` 进行子类化并实现以下方法来创建自定义层：

- build：创建层的权重。使用 add_weight 方法添加权重。
- call：定义前向传播。
- compute_output_shape：指定在给定输入形状的情况下如何计算层的输出形状。
- 或者，可以通过实现 get_config 方法和 from_config 类方法序列化层。

自定义层还是很有必要的。

### 回调 callbacks

回调是传递给模型的对象，用于在训练期间自定义该模型并扩展其行为。您可以编写自定义回调，也可以使用包含以下方法的内置 tf.keras.callbacks：

- tf.keras.callbacks.ModelCheckpoint：定期保存模型的检查点。
- tf.keras.callbacks.LearningRateScheduler：动态更改学习速率。
- tf.keras.callbacks.EarlyStopping：在验证效果不再改进时中断训练。
- tf.keras.callbacks.TensorBoard：使用 TensorBoard 监控模型的行为。

### 保存和恢复

#### 保存权重

使用 `tf.keras.Model.save_weights` 保存并加载模型的权重。使用 `tf.keras.Model.load_weights` 加载权重。

#### 保存配置

当然我们也可以选择只保存配置。相应 API.

```
json_str = model.to_json()
new_model = tf.keras.models.model_from_json(json_str)

yaml_str = model.to_yaml()
fresh_model = tf.keras.models.model_from_yaml(yaml_str)
```

#### 保存整个模型

```
model.save('my_model.h5')
```

## Eager Execution

Eager Execution 是一种命令式编程环境，可立即评估操作。

## 分布式

### Estimator

Estimator API 用于针对分布式环境训练模型。它适用于一些行业使用场景，例如用大型数据集进行分布式训练并导出模型以用于生产。

### 多个 GPU

tf.keras 模型可以使用 `tf.contrib.distribute.DistributionStrategy` 在多个 GPU 上运行。此 API 在多个 GPU 上提供分布式训练，几乎不需要更改现有代码。